In [1]:
import numpy as np
import cv2

In [2]:
net = cv2.dnn.readNetFromCaffe('MobileNetSSD_deploy.prototxt.txt', 'MobileNetSSD_deploy.caffemodel')

In [5]:
categories = { 0: 'background', 1: 'aeroplane', 2: 'bicycle', 3: 'bird', 4: 'boat', 5: 'bottle', 6: 'bus', 
              7: 'car', 8: 'cat', 9: 'chair', 10: 'cow', 11: 'diningtable', 12: 'dog', 13: 'horse', 
              14: 'motorbike', 15: 'person', 16: 'pottedplant', 17: 'sheep', 18: 'sofa', 19: 'train', 20: 'tvmonitor'}
# defined in list also
classes =  ["background", "aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair", "cow", 
            "diningtable",  "dog", "horse", "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]

In [12]:
def count_to_speed(count):
    if count < 1:
        speed = 20
    if count >= 1 and count < 3:
        speed = 15
    if count >= 3 and count < 5:
        speed = 10
    if count >=5:
        speed = 5
    return speed

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
        ret, frame = cap.read()
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        (h, w) = image.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 0.007843, (300, 300), 127.5)
        net.setInput(blob)
        detections = net.forward()
        
        colors = np.random.uniform(255, 0, size=(len(categories), 3))
        count = 0
        for i in np.arange(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.2:
                idx = int(detections[0, 0, i, 1])
                if idx == 15:
                    count += 1
                    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                    (startX, startY, endX, endY) = box.astype("int")
                    label = "{}: {:.2f}%".format(classes[idx], confidence * 100) 
                    cv2.rectangle(image, (startX, startY), (endX, endY), colors[idx], 2)     
                    y = startY - 15 if startY - 15>15 else startY + 15     
                    cv2.putText(image, label, (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[idx], 2)
                    cv2.putText(image, "number of people:{}".format(count), 
                                (0, 10),cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[15], 2)
                    speed = count_to_speed(count)
                    cv2.putText(image, "recomended speed:{}".format(speed), 
                                (0, 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[15], 2)
                
        cv2.imshow("Output", image)

        if cv2.waitKey(10) & 0xFF == ord('0'):
            break
cap.release()
cv2.destroyAllWindows()